In [2]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

from tqdm import tqdm

C:\Users\laura\AppData\Local\Temp\ipykernel_22032\101696790.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
#Creo la BBDD:

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')


mycursor = cnx.cursor()
try:
    mycursor.execute("CREATE DATABASE proyecto_cine")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

----
**API**

In [4]:
# Creo la tabla para las películas

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


crear_tabla = """
CREATE TABLE IF NOT EXISTS peliculas (
    Genero VARCHAR(255),
    Tipo VARCHAR(255),
    Titulo VARCHAR(255),
    Anio_de_estreno INT,
    Mes_de_estreno FLOAT,
    ID VARCHAR(255) PRIMARY KEY
)
"""

# Ejecuta la instrucción SQL para crear la tabla
try:
    mycursor.execute(crear_tabla)
    print("Tabla creada correctamente.")
except mysql.connector.Error as err:
    print("Error al crear la tabla:", err)

# Cierra el cursor y la conexión a la base de datos
mycursor.close()
cnx.close()

Tabla creada correctamente.


In [3]:
# Insertar información de las películas

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()

# Lee el archivo CSV
df = pd.read_csv("pelis_ordenadas.csv")
df.drop(df.columns[0], axis=1, inplace=True) #elimina la primera columna

# Define el rango de filas sobre el cual iterar
inicio = 0  
fin = 581694   

# Itera sobre el rango de filas especificado del DataFrame
for index in tqdm(range(inicio, fin), desc="Insertando registros"):
    row = df.iloc[index]  # Obtén la fila en el índice actual
    sql = "INSERT INTO peliculas (Genero, Tipo, Titulo, Anio_de_estreno, Mes_de_estreno, ID) VALUES (%s, %s, %s, %s, %s, %s)"
    # Convierte el año de estreno y el mes a tipo int
    año_de_estreno = int(row['Año de estreno']) if not pd.isnull(row['Año de estreno']) else None
    mes_de_estreno = float(row['Mes de estreno']) if not pd.isnull(row['Mes de estreno']) else None
    val = (row['Genero'], row['Tipo'], row['Titulo'], año_de_estreno, mes_de_estreno, row['ID'])
    try:
        mycursor.execute(sql, val)
        cnx.commit()  # Confirma la transacción
    except mysql.connector.Error as err:
        cnx.rollback()  # Revierte la transacción si hay un error
        print("Error al insertar registro:", err)

# Cierra el cursor y la conexión a la base de datos
mycursor.close()
cnx.close()

Insertando registros:   0%|          | 1114/581694 [00:02<17:35, 550.15it/s]

Error al insertar registro: 1062 (23000): Duplicate entry 'tt0249588' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0255189' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0255019' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0256375' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0259023' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0231956' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0232079' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0236027' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0233600' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0239655' for key 'peliculas.PRIMARY'
Error al insertar re

Insertando registros:   0%|          | 1225/581694 [00:02<18:05, 534.99it/s]

Error al insertar registro: 1062 (23000): Duplicate entry 'tt0179987' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0181836' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0183869' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0184894' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0113092' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0213127' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0212346' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0211938' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0212879' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0214683' for key 'peliculas.PRIMARY'
Error al insertar re

Insertando registros:   0%|          | 1331/581694 [00:02<20:12, 478.77it/s]

...

Error al insertar registro: 1062 (23000): Duplicate entry 'tt9486348' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt4794734' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0344499' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0347512' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0347547' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0347757' for key 'peliculas.PRIMARY'


Insertando registros:   0%|          | 1536/581694 [00:03<19:17, 501.35it/s]

Error al insertar registro: 1062 (23000): Duplicate entry 'tt0311413' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0311582' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0336675' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0337134' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0343388' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0325181' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0810396' for key 'peliculas.PRIMARY'
Error al insertar registro: 1062 (23000): Duplicate entry 'tt0814243' for key 'peliculas.PRIMARY'


Insertando registros:   0%|          | 1538/581694 [00:03<19:13, 503.09it/s]


KeyboardInterrupt: 

----
**IMDB Y ROTTEN**

In [5]:
# Creo la tabla para IMDB

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


crear_tabla = """
CREATE TABLE IF NOT EXISTS peliculas_info (
    Titulo VARCHAR(255),
    ID VARCHAR(255),
    Puntuacion_IMDB VARCHAR(255),
    Direccion VARCHAR(255),
    Guionistas VARCHAR(255),
    Argumento TEXT,
    Duracion VARCHAR(255),
    Puntuacion_Rotten VARCHAR(255),
    FOREIGN KEY (ID) REFERENCES peliculas (ID) ON DELETE CASCADE ON UPDATE CASCADE
)
"""
# Puntuacion_IMDB, Direccion, Guionistas, Argumento, Duracion, Nombre_de_la_pelicula

try:
    mycursor.execute(crear_tabla)
    print("Tabla creada correctamente.")
except mysql.connector.Error as err:
    print("Error al crear la tabla:", err)


mycursor.close()
cnx.close()

Tabla creada correctamente.


In [31]:
#Insertar información IMDB

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


df = pd.read_csv("2007_laura_films_total.csv")
df.drop(df.columns[0], axis=1, inplace=True)


inicio = 0  
fin = 10   


for index in tqdm(range(inicio, fin), desc="Insertando registros"):
    row = df.iloc[index]  
    sql = "INSERT INTO info_imdb2 (Puntuacion_IMDB, Direccion, Guionistas, Argumento, Duracion, Nombre_pelicula) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
   
    try:
        mycursor.execute(sql, (str(row['Titulo']), str(row['ID']), str(row['Puntuacion_IMDB']), str(row['Dirección']), str(row['Guionistas']), str(row['Argumento']), str(row['Duración']), str(row['Puntuacion_Rotten']),))
        cnx.commit()  
    except mysql.connector.Error as err:
        cnx.rollback()  
        print("Error al insertar registro:", err)


mycursor.close()
cnx.close()

Insertando registros: 100%|██████████| 10/10 [00:00<00:00, 300.21it/s]


-----
**ACTORES**

In [35]:
# Creo la tabla actores

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


crear_tabla = """
CREATE TABLE IF NOT EXISTS info_actores (
    Nombre VARCHAR(255),
    Nacimiento VARCHAR(255),
    Conocido VARCHAR(255),
    Que_hace TEXT,
    Premios VARCHAR(255),
    Oscar VARCHAR(255)
)
"""
# Nombre,Nacimiento,Conocido,Que hace,Premios,Oscar

try:
    mycursor.execute(crear_tabla)
    print("Tabla creada correctamente.")
except mysql.connector.Error as err:
    print("Error al crear la tabla:", err)


mycursor.close()
cnx.close()

Tabla creada correctamente.


In [37]:
#Insertar información IMDB

cnx = mysql.connector.connect(
    user='root', password='AlumnaAdalab',
    host='127.0.0.1', database='proyecto_cine'
)

mycursor = cnx.cursor()


df = pd.read_csv("df_actores_info_42826_45826.csv")
df.drop(df.columns[0], axis=1, inplace=True)

inicio = 11  
fin = 15 


for index in tqdm(range(inicio, fin)):
   
    row = df.iloc[index]
    
    sql = "INSERT INTO info_actores (Nombre, Nacimiento, Conocido, Que_hace, Premios, Oscar) VALUES (%s, %s, %s, %s, %s, %s)"
    
    try:
        mycursor.execute(sql, (str(row['Nombre']), str(row['Nacimiento']), str(row['Conocido']), str(row['Que hace']), str(row['Premios']), str(row['Oscar'])))
        cnx.commit() 
    except mysql.connector.Error as err:
        cnx.rollback()  
        print("Error al insertar registro:", err)


mycursor.close()
cnx.close()

100%|██████████| 4/4 [00:00<00:00, 442.20it/s]
